In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.insert(0, '../../')
from src.encode_data import *
from src.midi_data import *
from src.data_sources import process_all, arr2csv
from src.midi_transform import *
from src.fastai_data import *
from src.unilm import S2SFileProcessor, S2SPreloader

In [3]:
import traceback
import time

## Standardize and reformat raw midi files before encoding to text
- Transform key to C major
- Remove unused instruments
- Combine multiple tracks with the same instrument into a single part
- Melody, Piano, String

### Load midi data

In [4]:
version = 'v15'
data_path = Path('data/midi')
version_path = data_path/version

In [5]:
import pandas as pd

In [6]:
# out_dir = 'midi_encode'
# duet_only = False
out_dir = 's2s_encode'
duet_only = True

In [7]:
source_dir = 'midi_sources'
source_csv = version_path/'metadata'/f'{source_dir}.csv'
out_csv = version_path/out_dir/f'{out_dir}.csv'
out_csv.parent.mkdir(parents=True, exist_ok=True)
source_csv, out_csv

(PosixPath('data/midi/v15/metadata/midi_sources.csv'),
 PosixPath('data/midi/v15/s2s_encode/s2s_encode.csv'))

In [8]:
# num_comps = 2 # note, duration
cutoff = 5 # max instruments
min_variation = 3 # minimum number of different midi notes played
# max_dur = 128

### Encoding midi to numpy

In [9]:
df = pd.read_csv(source_csv); df.head()

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (3,4,7,9,12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ht_time_signature,ht_offset,midi,section,parts,ht_bpm,title,midi_title,artist,song_url,genres,source,ht_key,md5,mxl,ht_mode
0,4.0,0.0,midi_sources/hooktheory/pianoroll/w/wayne-shar...,chorus,"intro,chorus",128.0,yu-gi-oh-theme-song,yu-gi-oh3,wayne-sharpe,https://www.hooktheory.com/theorytab/view/wayn...,NaN,hooktheory,C,bf1f29e5ff84e3e93e37fb873bfb590e,NaN,1.0
1,3.0,0.0,midi_sources/hooktheory/pianoroll/w/wayne-shar...,intro,"intro,chorus",85.0,yu-gi-oh-theme-song,yu-gi-oh,wayne-sharpe,https://www.hooktheory.com/theorytab/view/wayn...,NaN,hooktheory,C,055f80ad67f64edb14a85ca8fbfe8c29,NaN,1.0
2,4.0,-5.0,midi_sources/hooktheory/pianoroll/w/what-a-day...,chorus,chorus,96.0,kiefer,kiefer,what-a-day,https://www.hooktheory.com/theorytab/view/what...,Jazz,hooktheory,D,197f96f5d181f6ce1e2c5ab04ac1ff87,NaN,6.0
3,4.0,-5.0,midi_sources/hooktheory/pianoroll/w/whiteflame...,pre-chorus,"verse,pre-chorus,chorus",152.0,senbonzakura,senbonzakura - pre-Pre-Chorus,whiteflame,https://www.hooktheory.com/theorytab/view/whit...,"J-Pop,Pop",hooktheory,D,9e7ce13a35f1314423a9a6d5a5287a4a,NaN,6.0
4,4.0,-5.0,midi_sources/hooktheory/pianoroll/w/whiteflame...,verse,"verse,pre-chorus,chorus",152.0,senbonzakura,Senbonzakura,whiteflame,https://www.hooktheory.com/theorytab/view/whit...,"J-Pop,Pop",hooktheory,D,d5aaf79d0989222f1362f9f46c540a27,NaN,6.0


In [10]:
all_records = df.to_dict(orient='records'); len(all_records)

197182

In [12]:
def part_enc(chordarr, part):
    partarr = chordarr[:,part:part+1,:]
    # Part 3. Chord array to numpy
    seq = chordarr2seq(partarr)
    
    npenc = seq2npenc(seq)
    if (npenc[:,1] >= DUR_RANGE).any(): 
        print(f'npenc exceeds max {DUR_RANGE} duration:', input_path)
        return None
    
    # https://en.wikipedia.org/wiki/Scientific_pitch_notation - 88 key range - 21 = A0, 108 = C8
    if ((npenc[...,0] > VALTSEP) & ((npenc[...,0] < PIANO_RANGE[0]) | (npenc[...,0] >= PIANO_RANGE[1]))).any(): 
        print('npenc out of piano note range 12 - 116:', input_path)
        return None
    
    return npenc

In [13]:
def process_metadata(metadata):
    result = metadata.copy()
    
    # Part 1. Compress tracks/instruments
    if not isinstance(metadata.get('midi'), str): return None
    
    input_path = version_path/metadata['midi']
    extension = input_path.suffix.lower()
    if not input_path.exists(): 
        print('Input path does not exist:', input_path, metadata)
        return result
    
    # Get outfile and check if it exists
    out_file = Path(str(input_path).replace(f'/{source_dir}/', f'/{out_dir}/'))
    out_file = out_file.with_suffix('.npy')
    out_file.parent.mkdir(parents=True, exist_ok=True)
    if out_file.exists(): 
        result['numpy'] = str(out_file.relative_to(version_path))
        return result
    
    npenc = transform_midi(input_path)
    if npenc is None: return result
    np.save(out_file, npenc)
    result['numpy'] = str(out_file.relative_to(version_path))
    return result

In [14]:
def transform_midi(midi_file):
    input_path = midi_file
    
    if duet_only:
        try: 
            if num_piano_tracks(input_path) != 2: return None
        except Exception: return None
    
    try: input_file = compress_midi_file(input_path, min_variation=min_variation, cutoff=cutoff) # remove non note tracks and standardize instruments
    except Exception as e:
        print('Error parsing midi', input_path, e)
        return None
    if not input_file: return None
        
    # Part 2. Compress rests and long notes
    stream = file2stream(input_file) # 1.
    try:
        chordarr = stream2chordarr(stream, max_dur=DUR_RANGE-2, flat=False) # 2. max_dur = quarter_len * sample_freq (4). 128 = 8 bars
    except Exception as e:
        print('Could not encode to chordarr:', input_path, e)
#         print(traceback.format_exc())
        return None
    
    chord_trim = trim_chordarr_rests(chordarr)
    chord_short = shorten_chordarr_rests(chord_trim)
    delta_trim = chord_trim.shape[0] - chord_short.shape[0]
    if delta_trim > 300: 
        print(f'Removed {delta_trim} rests from {input_path}. Skipping song')
        return None
    chordarr = chord_short
    
    _,num_inst,_ = chordarr.shape
    if num_inst != 2: return None
    
    parts = [part_enc(chordarr, i) for i in range(num_inst)]
    
    for p in parts:
        if p is None: return None
    
    return np.array(parts)

In [23]:
# transform_midi(piano_file)
midi_mxl_file = version_path/'midi_sources/from_mxl/musescore/data/49143.mid'
input_file = midi_mxl_file
stream = file2stream(input_file) # 1.
chordarr = stream2chordarr(stream, max_dur=DUR_RANGE-2, flat=False)

In [24]:
chordarr.shape

(1021, 2, 128)

In [25]:
transform_midi(midi_mxl_file).shape

(2,)

In [26]:
def try_process_metadata(metadata):
    try:
        return process_metadata(metadata)
    except Exception:
#         print(traceback.format_exc())
        return None

In [27]:
# # sanity check
import random
for r in random.sample(all_records, 10):
    process_metadata(r)

In [28]:
def timeout_func(data, seconds):
    print("Timeout:", seconds, data.get('midi'))

In [29]:
processed = process_all(try_process_metadata, all_records, timeout=300, timeout_func=timeout_func)

Removed 320 rests from data/midi/v15/midi_sources/freemidi/genre-dance-eletric/Fatboy Slim - Right Here Right Now.mid. Skipping song
Removed 456 rests from data/midi/v15/midi_sources/freemidi/genre-pop/Sade - Siempre Hay Esperanza.mid. Skipping song
Removed 320 rests from data/midi/v15/midi_sources/freemidi/genre-pop/Shakira - Waka Waka.mid. Skipping song
Removed 544 rests from data/midi/v15/midi_sources/midiworld/named_midi/Tag_Team_-_Whoomp_There_It_Is.mid. Skipping song
Removed 512 rests from data/midi/v15/midi_sources/midiworld/named_midi/Aerosmith_-_Falling_in_Love_Is_Hard_on_my_Knees.mid. Skipping song
Could not encode to chordarr: data/midi/v15/midi_sources/midiworld/named_midi/Howlin_Wolf_-_Little_Red_Rooster.mid index 1147 is out of bounds for axis 0 with size 1147
Removed 2064 rests from data/midi/v15/midi_sources/midiworld/named_midi/Metallica_-_One.mid. Skipping song
Could not encode to chordarr: data/midi/v15/midi_sources/midiworld/named_midi/Jelly_Roll_Morton_-_Honky_Tonk

Removed 352 rests from data/midi/v15/midi_sources/lmd_clean/Michael George/Freedom 90.mid. Skipping song
Removed 3212 rests from data/midi/v15/midi_sources/lmd_clean/Gershwin/Rhapsody in Blue.mid. Skipping song
Removed 3460 rests from data/midi/v15/midi_sources/lmd_clean/LIGABUE/Ho messo via.mid. Skipping song
Timeout: 300 midi_sources/from_mxl/musescore/data/4453696.mid
Removed 348 rests from data/midi/v15/midi_sources/130k_reddit/O/O/o_spirit.mid. Skipping song
Could not encode to chordarr: data/midi/v15/midi_sources/130k_reddit/O/O/O'Scarrafone.mid index 1807 is out of bounds for axis 0 with size 1807
Timeout: 300 midi_sources/from_mxl/musescore/data/2119261.mid
Timeout: 300 midi_sources/from_mxl/musescore/data/1671046.mid
Timeout: 300 midi_sources/from_mxl/musescore/data/1188146.mid
Timeout: 300 midi_sources/from_mxl/musescore/data/2638111.mid
Timeout: 300 midi_sources/from_mxl/musescore/data/2279591.mid
Removed 476 rests from data/midi/v15/midi_sources/130k_reddit/N/N/nickelback-t

In [36]:
arr2csv(processed, out_csv); len(processed)

195350

In [15]:
df = pd.read_csv(out_csv); df.head()

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (5,7,8,10,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ht_offset,ht_bpm,ht_mode,midi,ht_time_signature,parts,title,section,song_url,md5,ht_key,midi_title,genres,source,artist,mxl,numpy
0,0.0,128.0,1.0,midi_sources/hooktheory/pianoroll/w/wayne-shar...,4.0,"intro,chorus",yu-gi-oh-theme-song,chorus,https://www.hooktheory.com/theorytab/view/wayn...,bf1f29e5ff84e3e93e37fb873bfb590e,C,yu-gi-oh3,NaN,hooktheory,wayne-sharpe,NaN,s2s_encode/hooktheory/pianoroll/w/wayne-sharpe...
1,0.0,85.0,1.0,midi_sources/hooktheory/pianoroll/w/wayne-shar...,3.0,"intro,chorus",yu-gi-oh-theme-song,intro,https://www.hooktheory.com/theorytab/view/wayn...,055f80ad67f64edb14a85ca8fbfe8c29,C,yu-gi-oh,NaN,hooktheory,wayne-sharpe,NaN,NaN
2,-5.0,96.0,6.0,midi_sources/hooktheory/pianoroll/w/what-a-day...,4.0,chorus,kiefer,chorus,https://www.hooktheory.com/theorytab/view/what...,197f96f5d181f6ce1e2c5ab04ac1ff87,D,kiefer,Jazz,hooktheory,what-a-day,NaN,s2s_encode/hooktheory/pianoroll/w/what-a-day/k...
3,-5.0,152.0,6.0,midi_sources/hooktheory/pianoroll/w/whiteflame...,4.0,"verse,pre-chorus,chorus",senbonzakura,pre-chorus,https://www.hooktheory.com/theorytab/view/whit...,9e7ce13a35f1314423a9a6d5a5287a4a,D,senbonzakura - pre-Pre-Chorus,"J-Pop,Pop",hooktheory,whiteflame,NaN,s2s_encode/hooktheory/pianoroll/w/whiteflame/s...
4,-5.0,152.0,6.0,midi_sources/hooktheory/pianoroll/w/whiteflame...,4.0,"verse,pre-chorus,chorus",senbonzakura,verse,https://www.hooktheory.com/theorytab/view/whit...,d5aaf79d0989222f1362f9f46c540a27,D,Senbonzakura,"J-Pop,Pop",hooktheory,whiteflame,NaN,s2s_encode/hooktheory/pianoroll/w/whiteflame/s...


In [16]:
len([f for f in df.numpy.values if isinstance(f, str)])

51520

In [17]:
from collections import Counter

In [18]:
df[df.numpy.notnull()]

,ht_offset,ht_bpm,ht_mode,midi,ht_time_signature,parts,title,section,song_url,md5,ht_key,midi_title,genres,source,artist,mxl,numpy
0,0.0,128.0,1.0,midi_sources/hooktheory/pianoroll/w/wayne-shar...,4.0,"intro,chorus",yu-gi-oh-theme-song,chorus,https://www.hooktheory.com/theorytab/view/wayn...,bf1f29e5ff84e3e93e37fb873bfb590e,C,yu-gi-oh3,NaN,hooktheory,wayne-sharpe,NaN,s2s_encode/hooktheory/pianoroll/w/wayne-sharpe...
2,-5.0,96.0,6.0,midi_sources/hooktheory/pianoroll/w/what-a-day...,4.0,chorus,kiefer,chorus,https://www.hooktheory.com/theorytab/view/what...,197f96f5d181f6ce1e2c5ab04ac1ff87,D,kiefer,Jazz,hooktheory,what-a-day,NaN,s2s_encode/hooktheory/pianoroll/w/what-a-day/k...
3,-5.0,152.0,6.0,midi_sources/hooktheory/pianoroll/w/whiteflame...,4.0,"verse,pre-chorus,chorus",senbonzakura,pre-chorus,https://www.hooktheory.com/theorytab/view/whit...,9e7ce13a35f1314423a9a6d5a5287a4a,D,senbonzakura - pre-Pre-Chorus,"J-Pop,Pop",hooktheory,whiteflame,NaN,s2s_encode/hooktheory/pianoroll/w/whiteflame/s...
4,-5.0,152.0,6.0,midi_sources/hooktheory/pianoroll/w/whiteflame...,4.0,"verse,pre-chorus,chorus",senbonzakura,verse,https://www.hooktheory.com/theorytab/view/whit...,d5aaf79d0989222f1362f9f46c540a27,D,Senbonzakura,"J-Pop,Pop",hooktheory,whiteflame,NaN,s2s_encode/hooktheory/pianoroll/w/whiteflame/s...
5,-5.0,152.0,6.0,midi_sources/hooktheory/pianoroll/w/whiteflame...,4.0,"verse,pre-chorus,chorus",senbonzakura,chorus,https://www.hooktheory.com/theorytab/view/whit...,e0c189ee753b30c4758d85211f13c189,D,Senbonzakura,"J-Pop,Pop",hooktheory,whiteflame,NaN,s2s_encode/hooktheory/pianoroll/w/whiteflame/s...
6,-1.0,108.0,1.0,midi_sources/hooktheory/pianoroll/w/wham/last-...,4.0,"intro,verse,chorus",last-christmas,verse,https://www.hooktheory.com/theorytab/view/wham...,38e38402443506e326b76536e8e327a0,Db,Last Christmas Verse,Holiday,hooktheory,wham,NaN,s2s_encode/hooktheory/pianoroll/w/wham/last-ch...
7,-1.0,108.0,1.0,midi_sources/hooktheory/pianoroll/w/wham/last-...,4.0,"intro,verse,chorus",last-christmas,chorus,https://www.hooktheory.com/theorytab/view/wham...,75d0251177c8c1fa9a02821299fa5ba8,Db,Last Christmas Chorus,Holiday,hooktheory,wham,NaN,s2s_encode/hooktheory/pianoroll/w/wham/last-ch...
8,-1.0,108.0,1.0,midi_sources/hooktheory/pianoroll/w/wham/last-...,4.0,"intro,verse,chorus",last-christmas,intro,https://www.hooktheory.com/theorytab/view/wham...,83d2a800f40aeca07e30e4718cda8fe5,Db,Last Christmas Intro,Holiday,hooktheory,wham,NaN,s2s_encode/hooktheory/pianoroll/w/wham/last-ch...
9,0.0,128.0,1.0,midi_sources/hooktheory/pianoroll/w/wham/freed...,4.0,chorus,freedom,chorus,https://www.hooktheory.com/theorytab/view/wham...,60fa29cfec107df27b053cf9708823d5,C,Freedom Chorus,NaN,hooktheory,wham,NaN,s2s_encode/hooktheory/pianoroll/w/wham/freedom...
11,5.0,86.0,1.0,midi_sources/hooktheory/pianoroll/w/wg-snuffy-...,4.0,instrumental,west-wing-suite,instrumental,https://www.hooktheory.com/theorytab/view/wg-s...,a856dff6c54398544c217104d047abe0,G,snuffy,NaN,hooktheory,wg-snuffy-walden,NaN,s2s_encode/hooktheory/pianoroll/w/wg-snuffy-wa...


In [19]:
Counter(df[df.numpy.notnull()].source.values)

Counter({'hooktheory': 17813,
         'freemidi': 28,
         'midiworld': 44,
         'ecomp': 2226,
         'cprato': 96,
         'classical_piano': 318,
         'classical_archives': 2350,
         'musescore': 6674,
         'wikifonia': 33,
         'lmd': 78,
         'reddit': 3917,
         'hooktheory_c': 17943})

In [20]:
len(df[df.numpy.notnull()].source.values)

51520

In [21]:
Counter(df.source.values)

Counter({'hooktheory': 19830,
         'freemidi': 5164,
         'midiworld': 4097,
         'ecomp': 2242,
         'cprato': 305,
         'classical_piano': 320,
         'classical_archives': 14546,
         'musescore': 10526,
         'wikifonia': 6345,
         'lmd': 13555,
         'reddit': 98396,
         'hooktheory_c': 20024})

## Convert to hooktheory databunch

In [22]:
def get_files(csv):
    files = csv['numpy']
    flist = [Path(version_path/f) for f in files.values if isinstance(f, str)]
    flist = [f for f in flist if f.exists()]
    return flist

In [25]:
def create_databunch(files, cache_name, batch_size=32, load_cached=False):
    if load_cached and (out_path/f'{cache_name}/train_ids.npy').exists():
        data = MusicDataBunch.load(out_path, bs=batch_size, cache_name=cache_name)
    else:
        ps = [S2SFileProcessor()]
        data = (MusicItemList(items=files, path=out_path, processor=ps)
                .split_by_rand_pct(0.01, seed=6)
                .label_const(label_cls=LMLabelList))
#         data.x._bunch = MusicDataBunch
        data = data.databunch(bs=batch_size, preloader_cls=S2SPreloader)
        data.save(cache_name)
    return data

In [31]:
out_path = version_path/out_dir

In [28]:
csv = df

In [48]:
hook_csv = csv.loc[csv.source.isin(['hooktheory'])]
hook_files = get_files(hook_csv); len(hook_files)
hook_data = create_databunch(hook_files, cache_name='tmp/hook')

In [49]:
hook_csv = csv.loc[csv.source.isin(['hooktheory_c'])]
hook_files = get_files(hook_csv); len(hook_files)
hook_data = create_databunch(hook_files, cache_name='tmp/hook_c')

In [58]:
len(hook_data.train_dl.dl.dataset)

17125

In [29]:
all_files = get_files(csv); len(all_files)
# all_data = create_databunch(all_files, cache_name='tmp/all')

51520

In [32]:
import random
sample_data = create_databunch(random.sample(all_files, 1000), cache_name='tmp/sample')

## Load data

In [103]:
single_tfm = partial(to_single_stream, vocab=vocab)
transpose_tfm = partial(rand_transpose, note_range=vocab.note_range, rand_range=(0,12))
load_data =  MusicDataBunch.load(path=out_path, cache_name='tmp/hook_c', preloader_cls=S2SPreloader, train_tfms=[single_tfm, transpose_tfm])

DLTFMS: None
You can deactivate this warning by passing `no_check=True`.


/home/ubuntu/fastai/fastai/basic_data.py:259: UserWarning: There seems to be something wrong with your dataset, for example, in the first batch can't access any element of self.train_ds.
Tried: 0,1,2,3,4...
  warn(warn_msg)


In [23]:
load_data =  MusicDataBunch.load(path=out_path, cache_name='tmp/hook_c', preloader_cls=S2SPreloader)

DLTFMS: None


In [24]:
load_data.one_batch()

(tensor([[  6, 273,   8,  ...,   1,   1,   1],
         [  6, 273,   8,  ..., 142,  88, 141],
         [  6, 273,   8,  ..., 145,  85, 141],
         ...,
         [  6, 273,   8,  ..., 141,  62, 141],
         [  6, 273,   8,  ..., 141,  54, 143],
         [  6, 273,   8,  ..., 143,  61, 143]]),
 tensor([[  5, 273,   8,  ..., 147,  69, 147],
         [  5, 273,   8,  ..., 149,   8, 149],
         [  5, 273,   8,  ..., 147,  61, 147],
         ...,
         [  5, 273,   8,  ..., 155,  59, 155],
         [  5, 273,   8,  ..., 147,  64, 147],
         [  5, 273,   8,  ..., 141,  68, 141]]))

In [ ]:
# ps = [S2SFileProcessor()]

# single_tfm = partial(to_single_stream, vocab=vocab)
# data = (MusicItemList(items=hook_files[:100], path=out_path, processor=ps, tfms=[single_tfm])
#         .split_by_rand_pct(0.01, seed=6)
#         .label_const(label_cls=LMLabelList))
# data.x._bunch = MusicDataBunch

In [78]:
# data.x.tfms = [single_tfm]

In [ ]:
data = data.databunch(bs=4, preloader_cls=S2SPreloader, train_tfms=[single_tfm])

In [80]:
out = data.train_dl.dl.dataset[0]

In [ ]:
data.one_batch()